In [1]:
import numpy as np
from numpy import exp,max, sqrt
from FERM import Binomial_Option

# Parameters used throughout the exercise as constants
S0 = 100            # Spot price
r = 0.02            # Fixed interest rate
sigma = 0.30        # Volatility
c = 0.01            # Dividend yield
n = 16

[Since:](http://www.math.nyu.edu/~cai/Courses/Derivatives/lecture8.pdf) *Given interest rate r > 0, it is never optimal to exercise an American call between ex-dividends dates or prior to maturity.*, it is better to exercise the call at maturity. i.e. $t=10$

# Homework III

### 1)
*Compute the price of an American call option with strike $K=110$ and maturity $T=.25$ years*

In [2]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

option1 = Binomial_Option(S0, r, sigma, n, T, c)
option1.option_price(K, form="call", style="american")
print("${:.2f}".format(option1.price))

$2.60


### 2)
Compute the price of an American put option with strike $K=110$ and maturity $T=.25$ years.

In [5]:
K = 110             # Strike Price
T = 0.25            # Time to maturity

put1 = Binomial_Option(S0, r, sigma, n, T, c)
put1.option_price(K, form="put", style="american")
print("${:.2f}".format(put1.price))

$12.37


### 3), 4)
*Is it ever optimal to exercise the put of Q2? If so, when it's the earliest?*

It is optimal to exercise an american put option at time $t$, whenever 
$$
\max\{C(t), E[C(t+1)]\} = C(t)
$$

That is, If the intrinsic value is greater or equal to the Risk-Neutral expected value, then it is optimal to exercise early

In [5]:
put1.optimal_early_exercise()

5

At $t=5$, if the option is *deep in the money*, then it is optimal, and the earliest, to exercise the american put.

### 5)
*Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?*

Recall the put-call parity:
$$
    P_E + S = C_E + K
$$

Since *1)* and *2)* are 'american' options, the put-call parity does **not** hold.

### 6)
Compute the fair value of an American call option with strike $K=110$ and maturity $n=10$ periods where the option is written on a **futures contract** that expires after 15 periods. The futures contract is on the same underlying security of the previous questions.

In [12]:
from FERM import *
putF = Binomial_Option(S0=100, r=0.02, sigma=0.30, n_periods=15, T=0.25, c=0.01)
putF.futures_option(110, fform="call", fstyle="american", time_maturity=10)
putF.price

1.6651388420235214

### 7)
What is the earliest time period in which you might want to exercise the American futures option of Question 6?

In [11]:
compare = np.array(putF.intrinsic_value_tree) - np.array(putF.present_val_tree)[1:]
for t, branch in enumerate(compare[::-1]):
    exercise = [round(x,5) > 0 for x in branch]
    print("t:{:>2}".format(t+1), end=" ")
    for v in exercise:
        print("{:>3}".format(v), end=" ")
    print()

ValueError: operands could not be broadcast together with shapes (25,) (10,) 

### 8)
*Compute the fair value of a chooser option which expires after n=10 periods. At expiration the owner of the chooser gets to choose (at no cost) a European call option or a European put option. The call and put each have strike K=100 and they mature 5 periods later, i.e. at n=15.*

Since the option is chosen up until $t=10$, we must compute the possible paths of the stock at $t=10$. Once knowing all the possible paths of the stock at that time, we must compute a 5 period option from both the call and put options. For each possible path, take the biggest between the two.  Finally, roll back the binomial tree with the computed prices down to $t=0$

In [28]:
chooser = Binomial_Option(100, r, sigma, 11, T=0.25, c=c)

prices = []

for s10 in chooser.tree[-1]:
    put = Binomial_Option(s10, r, sigma, 16, T=0.25, c=c)
    put.option_price(100, form="put", style="european")
    
    call = Binomial_Option(s10, r, sigma, 16, T=0.25, c=c)
    call.option_price(100, form="call", style="european")

    prices.append(np.max([put.price, call.price]))
    
chooser.tree[-1] = np.array(prices)
chooser.option_price(K=0)

print("${:.2f}".format(chooser.price))

$14.58


In [29]:
chooser_price = 0

chooser = Binomial_Option(100, r, sigma, 10, T=0.25, c=c)
chooser.option_price(100, form="call", style="european")
chooser_price += chooser.price

d = chooser.rate
chooser = Binomial_Option(100*exp(-c*0.25/10), r, sigma, 10, T=0.25, c=c)
chooser.option_price(100*exp(-r*0.25/10), form="put", style="european")
chooser_price += chooser.price

print(chooser_price)

11.6379807452
